# QUEEN Cloning Script for a Large Catalog of Plasmids
### Samuel King | Yachie Lab | Last updated: January 2023

This is a QUEEN script for constructing plasmids by Digestion-Ligation, Gibson Assembly, and Golden Gate Assembly for the plasmids described in the CloneSelect paper.

### Setting up QUEEN

* I recommend using Anaconda as a distribution and coding your QUEEN scripts in JupyterLab.
* This tutorial https://biocircuits.github.io/chapters/00_setting_up_python_computing_environment.html describes very clearly how to set up your environment with Anaconda.
* Write out the exact cloning plan for your desired plasmid step-by-step, including template plasmid names, primer names, enzyme names, etc., before starting your QUEEN script.
* If designing many plasmids, a spreadsheet is useful to carry the information. Then, you can code your functions to work with the spreadsheet items.
* The full tutorial for QUEEN can be found here: https://github.com/yachielab/QUEEN

### After setting up Anaconda and getting JupyterLab going, start your script like this:

In [1]:
%matplotlib agg

import sys
from QUEEN.queen import *
set_namespace(globals())
from QUEEN import cutsite as cs
import re
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
from Bio import GenBank, SeqIO
import time


In [2]:
#Load primers, plasmids, and assemblies
file_path = '../QUEEN/Assemblies Information Table_revise_SI1.xlsx'
oligos = pd.read_excel(file_path, sheet_name=0)
plasmids = pd.read_excel(file_path, sheet_name=1)
digestion_ligation_assemblies = pd.read_excel(file_path, sheet_name=2)
golden_gate_assemblies = pd.read_excel(file_path, sheet_name=3)
gibson_assemblies = pd.read_excel(file_path, sheet_name=4)
processes = pd.read_excel(file_path, sheet_name=5)

#Generate oligo QUEEN object list
oligo_list = []
i = 0
while i < len(oligos):
    primer = QUEEN(seq = oligos['seq'][i], product = oligos['primer_id'][i])
    oligo_list.append(primer)
    i = i+1


# this takes HTTP requests
plasmid_list = []
i = 0
while i < len(plasmids):
    plasmid = QUEEN(record = str(plasmids['record'][i]), dbtype = str(plasmids['dbtype'][i]), product = str(plasmids['plasmid_id'][i]))
    plasmid_list.append(plasmid)
    time.sleep(1)

    i = i+1

oligo_name_list = []
i = 0
while i < len(oligos['primer_id']):
    name = oligos['primer_id'][i]
    oligo_name_list.append(name)
    i = i+1

plasmid_name_list = []
i = 0
while i < len(plasmids['plasmid_id']):
    name = plasmids['plasmid_id'][i]
    plasmid_name_list.append(name)
    i = i+1


In [3]:
plasmid_list
oligo_list 
plasmid_name_list

['pSI_356_v2', 'pSI-894', 'pLV-CS-282_v2', 'pLV-CS-281_v4', 'lentiGuide_Puro']

### Golden_Gate_Assembly_2 Template
gRNA and reporter construction with BsmBI Golden Gate Assembly


In [8]:
#Set up the data for plasmids constructed by Golden_Gate_Assembly_2
assembly_method = golden_gate_assemblies['assembly_method'] #extract the assembly_method column from the golden_gate sheet
golden_gate_assembly_2 = ['Golden_Gate_Assembly_2'] #set up for extracting rows with Golden_Gate_Assembly_2
golden_gate_assembly_2 = golden_gate_assemblies.query('@golden_gate_assembly_2 in assembly_method') #extract only rows containing Golden_Gate_Assembly_2
golden_gate_assembly_2.index = range(len(golden_gate_assembly_2)) #re-index the extracted list to be in the order of only the rows containing Golden_Gate_Assembly_2

def find_primer_binding_str(primer_type,  #'FW' or 'RV'
                            primer,       #QUEEN object
                            template      #QUEEN object
                           ):
    """
    Given a type of primer ('FW' or 'RV'), the primer, and template sequence, finds the initial binding site of the primer in the template sequence, whether the primer has an overhang or not.
    Returns the string sequence of the portion of the primer that binds the template sequence.
    Requires minimum 18bp annealing between primer and template. The 3' end of the binding primer must anneal.
    """
    # Note: re.search cannot search in the 3' to 5' direction on a QUEEN DNA object, so the function will not work for Rv primers unless the template is flipped
    # For example:
      # match1 = re.search('hell', 'hello') #Finds match='hell'
      # print(match1)
      # match2 = re.search('lleh', 'hello') #Finds None
      # print(match2)
    
    n = len(primer.seq)                  #length of primer
    i=-18                                #starting point of i
    flippedtemplate = flipdna(template)  #reverse complement of template
    binding_site = []
    
    
    if primer_type == 'FW':                                 #for searching templates with Fw primer
        while i <= n:
            match = re.search(primer.seq[i:], template.seq) #search for each character in the primer string until the maximum amount of the primer matches the template string
            binding_site.append(match)                      #add matching string to the list binding_site
            if match == None:                               #end loop when there are no more matches
                break
            elif -i == n:                                   #end loop when there are no is no more primer string to iterate on
                break
            i = i-1
    elif primer_type == 'RV':                               #for searching templates with Rv primer; template must be reverse complement because re can't search on dsDNA like QUEEN can
        while i <= n:
            match = re.search(primer.seq[i:], flippedtemplate.seq)
            binding_site.append(match)
            if match == None:
                break
            elif -i == n:
                break
            i = i-1
            
    if len(binding_site) == 1 and binding_site[-1] == None:  #if there is no binding sequence, the binding_site list will only contain 1 NoneType object
        return 'No primer binding site found.'               #notify that no binding sequence for the primer was found in the template
    elif binding_site[-1] == None:                           #the while loop ends the binding_site list with a NoneType object if the primer has an overhang
        return binding_site[-2].group()                      #show the last matching binding site for a primer with an overhang
    else:
        return binding_site[-1].group()                      #show the last matching binding site for a primer with no overhang
        

def find_primer_binding_primer_str(fw_primer,  #QUEEN object
                                   rv_primer   #QUEEN object
                                  ):
    """
    Searches for binding sites between two primers (useful for template-free PCR or annealing of two ssDNA oligos).
    Returns string of the binding portion of the primer that anneals to the other primer. 3' end of primer must anneal.
    If primers anneal and are equal size, returns string of the binding portion of the primer provided second.
    Requires minimum 18-bp annealing between two oligos.
    """
    
    #The 'RV' feature must be set for every search because by design, every template (whether it's a Fw primer or Rv primer),
    #will be antisense to the query, unlike when we're searching on a plasmid, which has a sense strand and an antisense strand
    
    if len(fw_primer.seq) > len(rv_primer.seq):                         #The smaller primer needs to be the query and the larger primer needs to be the template searched on.
        return find_primer_binding_str('RV', rv_primer, fw_primer)      
    elif len(fw_primer.seq) < len(rv_primer.seq):
        return find_primer_binding_str('RV', fw_primer, rv_primer)
    elif len(fw_primer.seq) == len(rv_primer.seq):
        return find_primer_binding_str('RV', fw_primer, rv_primer)


def typeIIS_digest_backbone(dna,  #QUEEN object
                            re,   #str
                            pn,   #str
                            pd    #str
                           ):
    """
    Given a QUEEN DNA sequence for a backbone, finds the Type II-S restriction site(s) and digests at the found sites.
    User may specify a process name (pn) and process description (pd) as strings.
    Returns the cropdna backbone object. Also provides length and the object's topology (linear in this case).
    """
    from QUEEN import cutsite #Import a restriction enzyme library 
    
    sites = dna.searchsequence(cutsite.lib[re])
    fragments = cropdna(dna, *sites, pn=pn, pd=pd) #Produces the backbone fragment
    return fragments #Keep the backbone fragment

def anneal_oligos_object(fw_primer,  #QUEEN object
                         rv_primer,  #QUEEN object
                         pn,         #str
                         pd          #str
                        ):
    """
    Given Fw and Rv primers, creates a new fragment through ssDNA annealing of the two primers to each other.
    *Requirement: Primers must anneal with at least 18 bp of binding.
    Leaves overhangs on either, neither, or both sides depending on the design of the two oligos.
    User may specify a process name (pn) and process description (pd) as strings.
    Returns the QUEEN object with name " ". Also provides length and the object's topology (linear in this case).
    """
    #
    # To build dsDNAs from two ssDNA oligos annealing (but not extending), we must consider the nine possible combinations of oligos:
    #
    #               Case 1           Case 2                           Case 3                           Case 4                           Case 5
    # Fw primer:    5'-anneal-3'     5'-overhang-anneal-overhang-3'   5'----------anneal----------3'   5'-overhang-anneal----------3'   5'----------anneal-overhang-3'
    # Rv primer:    3'-anneal-5'     3'----------anneal----------5'   3'-overhang-anneal-overhang-5'   3'----------anneal-overhang-5'   3'-overhang-anneal----------5'
    #                  Frag.1            Frag.2          Frag.3           Frag.4          Frag.5   
    #
    #                                Case 6                           Case 7                           Case 8                           Case 9
    # Fw primer:                     5'-overhang-anneal-3'            5'----------anneal-3'            5'-anneal-overhang-3'            5'-anneal----------3'
    # Rv primer:                     3'----------anneal-5'            3'-overhang-anneal-5'            3'-anneal----------5'            3'-anneal-overhang-5'
    #
    #
    # With the information of two primer sequences we must generate 5 unique fragments that will exist or not exist depending on which IF statement is satisified.
    # Then, we will concatenate fragments depending on the condition satisfied.
    #
    # Approach: Generate each fragment, then concatenate them on the ds_anneal string, and the product should be one of cases 1-9.
    # If an annealed oligo doesn't exist, it should notify the user.
    #
    
    fw_primer_str = fw_primer.seq                                            #String of Fw primer sequence
    rv_primer_str = rv_primer.seq                                            #String of Rv primer sequence
    
    if fw_primer_str == rv_primer_str:                                       #Function can't run if oligos are exactly identical
        return 'Identical ssDNAs cannot anneal. Please revise your entry.'
    elif len(fw_primer_str) < 18:                                            #Function can't run if oligos have <18 bp annealing.
        return 'Oligos could not anneal because at least one is too short. Please revise your entry.'
    elif len(rv_primer_str) < 18:                                            #Function can't run if oligos have <18 bp annealing.
        return 'Oligos could not anneal because at least one is too short. Please revise your entry.'
    else:
        pass
    
    fw_overhang_list = []                                                    #Empty list for Fw overhang(s)
    rv_overhang_list = []                                                    #Empty list for Rv overhang(s)
    fw_fragment_list = []                                                    #Empty list for Fw overhang fragment(s)
    rv_fragment_list = []                                                    #Empty list for Fw overhang fragment(s)
    
    #Generate Frag.1
    anneal = find_primer_binding_primer_str(fw_primer, rv_primer)            #Finds the annealing site between the two primers
    if fw_primer_str.__contains__(anneal) == True:
        ds_anneal = generate_dsdna(anneal)                                   #Generates the double-stranded sequence of the binding sites of the two primers upon annealing
    elif fw_primer_str.__contains__(anneal) == False:                        #Error for Case 5 occurs here because the 3' end of the Rv primer doesn't anneal
        ds_anneal = generate_dsdna(flipdna(anneal))                          #Generates the double-stranded sequence of the binding sites of the two primers upon annealing
    
    
    #Generate Frag.2 and Frag.3
    overhangs_fw_primer = primer_overhangs(fw_primer, anneal)
    five_fw = five_prime_overhang(overhangs_fw_primer)                       #Provides the 5' overhang of the Fw primer (if any)
    fw_overhang_list.append(five_fw)
    three_fw = three_prime_overhang(overhangs_fw_primer)                     #Provides the 3' overhang of the Fw primer (if any)
    fw_overhang_list.append(three_fw)
    
    #Position 0 of fw_fragment_list will correspond to Frag.2
    #Position 1 of fw_fragment_list will correspond to Frag.3
    i=0
    while i < 2:
        if fw_overhang_list[i] != '':
            fragment = QUEEN(seq=fw_overhang_list[i] + '/' + ('-' * len(fw_overhang_list[i])))
            fw_fragment_list.append(fragment)
        elif fw_overhang_list[i] == '':
            fw_fragment_list.append('No overhang')
        elif i == 2:
            break
        i = i+1
    
    
    #Generate Frag.4 and Frag.5
    overhangs_rv_primer = primer_overhangs(rv_primer, anneal)
    five_rv = five_prime_overhang(overhangs_rv_primer)                       #Provides the 5' overhang of the Rv primer (if any)
    rv_overhang_list.append(five_rv)
    three_rv = three_prime_overhang(overhangs_rv_primer)                     #Provides the 3' overhang of the Rv primer (if any)
    rv_overhang_list.append(three_rv)
    
    #Position 0 of rv_fragment_list will correspond to Frag.4
    #Position 1 of rv_fragment_list will correspond to Frag.5
    i=0
    while i < 2:
        if rv_overhang_list[i] != '':
            fragment = QUEEN(seq=rv_overhang_list[i] + '/' + ('-' * len(rv_overhang_list[i])))
            rv_fragment_list.append(fragment)
        elif rv_overhang_list[i] == '':
            rv_fragment_list.append('No overhang')
        elif i == 2:
            break
        i = i+1

    
    if anneal != 'No primer binding site found.':
        
        if fw_fragment_list[0] != 'No overhang':    #Add upstream Fw overhang if possible
            frag2 = QUEEN(seq=fw_fragment_list[0].seq + ds_anneal.seq[0] + '/' + '-' * len(fw_fragment_list[0].seq) + generate_complement(ds_anneal.seq[0]))
            ds_anneal_left_trim = cropdna(ds_anneal, 1, len(ds_anneal.seq))
            anneal_left = joindna(frag2, ds_anneal_left_trim, pn=pn, pd=pd)
            
        elif rv_fragment_list[1] != 'No overhang':  #Add upstream Rv overhang if possible
            frag4 = QUEEN(seq=('-' * len(rv_fragment_list[1].seq) + ds_anneal.seq[0] + '/' + rv_fragment_list[1].seq[::-1] + generate_complement(ds_anneal.seq[0])))
            ds_anneal_left_trim = cropdna(ds_anneal, 1, len(ds_anneal.seq))
            anneal_left = joindna(frag4, ds_anneal_left_trim, pn=pn, pd=pd)
            
        else:
            anneal_left = ds_anneal
    
        if fw_fragment_list[1] != 'No overhang':    #Add downstream Fw overhang if possible
            frag3 = QUEEN(seq=ds_anneal.seq[-1] + fw_fragment_list[1].seq + '/' + generate_complement(ds_anneal.seq[-1]) + ('-' * len(fw_fragment_list[1].seq)))
            ds_anneal_right_trim = cropdna(anneal_left, 0, (len(anneal_left.seq) - 1))
            annealed_oligo = joindna(ds_anneal_right_trim, frag3, pn=pn, pd=pd)
            return annealed_oligo
            
        elif rv_fragment_list[0] != 'No overhang':  #Add downstream Rv overhang if possible
            frag5 = QUEEN(seq= ds_anneal.seq[-1] + ('-' * len(rv_fragment_list[0].seq)) + '/' + generate_complement(ds_anneal.seq[-1]) + rv_fragment_list[0].seq[::-1])
            ds_anneal_right_trim = cropdna(anneal_left, 0, (len(anneal_left.seq) - 1))
            annealed_oligo = joindna(ds_anneal_right_trim, frag5, pn=pn, pd=pd)
            return annealed_oligo
        
        elif fw_fragment_list[0] != 'No overhang':
            return anneal_left
            
        elif rv_fragment_list[1] != 'No overhang':
            return anneal_left
        
        else:
            annealed_oligo = ds_anneal
            return annealed_oligo
            
    else:
        'Oligos could not anneal.'
    

def generate_dsdna(dna  #str
                  ):
    """
    Given a single-stranded string of DNA in UPPERCASE letters only, returns the double-stranded QUEEN object.
    """
    sense = dna                                  #sense strand of dna
    antisense = generate_complement(dna)         #antisense strand of dna
    concatenate = sense + '/' + antisense        #concatenate into one string for QUEEN to recognize the object
    dsdna = QUEEN(seq=concatenate)               #generate dsDNA
    return dsdna                                 #print dsDNA


def generate_complement(dna #str
                       ):
    """
    Given a string of DNA in UPPERCASE letters only, returns complement of the DNA (not the reverse complement).
    """
    complementary_dna = dna.replace('A', 't').replace('T', 'a').replace('G', 'c').replace('C', 'g')
    return complementary_dna


def primer_overhangs(primer, #QUEEN object
                     anneal  #str
                    ):
    """
    Given a primer and its known annealing sequence to another primer (*required to anneal*), determines the overhangs on its 5' and 3' ends (if any).
    Returns the overhangs in the format 'NNN|NNN', where '|' separates the 5' overhang (left of the | ) and the 3' overhang (right of the bar | ).
    There can be 0 or more N's on either side of the '|'.
    Helper function for anneal_oligos.
    """
    p = primer.seq
    
    if p.__contains__(anneal) == True:
        return p.replace(anneal, '|')
    elif p.__contains__(anneal) == False:
        return p.replace(flipdna(anneal), '|')
    
def five_prime_overhang(string,
                       ):
    """
    Given a string of format 'NNN|NNN' where N can represent any nucleotide or absence of a nucleotide, returns the 5' side of the string (characters before the '|').
    Helper function for anneal_oligos.
    """ 
    i = 0
    string_new = ''
    n = len(string)
    
    while i <= n:
        string_new = string_new + string[i]
        if string[i] == '|':
            break
        elif i == n:
            break
        i = i+1
        
    return string_new[:i]

def three_prime_overhang(string,
                        ):
    """
    Given a string of format 'NNN|NNN' where N can represent any nucleotide or absence of a nucleotide, returns the 3' side of the string (characters after the '|').
    Helper function for anneal_oligos.
    """ 
    i = -1
    string_new = ''
    n = len(string)
    
    while i <= n:
        string_new = string[i] + string_new
        if string[i] == '|':
            break
        elif i == n:
            break
        i = i-1
    
    j = i+1 
    if string_new == '|':   #When '|' is the first character that i sees, then string_new becomes '|' and the loop ends. We remove this to give '' in such cases.
        return ''
    else:
        return string_new[j:]
    

## Golden Gate Assembly Process
For gRNA and CRISPRa reporters, used for revised experiments



In [5]:
#Set assembly steps

#Step 1
pn1 = processes['pn1'][3]
pd1 = processes['pd1'][3]
#Step 2
pn2 = processes['pn2'][3]
pd2 = processes['pd2'][3]
#Step 3
pn3 = processes['pn3'][3]
pd3 = processes['pd3'][3]



In [6]:
#Step 1: Digest backbone

digested_backbones = []
i = 0
while i < len(golden_gate_assembly_2):
    index_backbone = plasmid_name_list.index(golden_gate_assembly_2['backbone_id'][i])
    re1 = golden_gate_assembly_2['re1'][i]
    digested_backbone = typeIIS_digest_backbone(plasmid_list[index_backbone], re1, pn1, pd1)
    digested_backbones.append(digested_backbone)
    i = i+1

In [9]:
#Step 2: Anneal ssDNAs

annealed_DNA = []
i = 0
while i < len(golden_gate_assembly_2):
    index_fw = oligo_name_list.index(golden_gate_assembly_2['fw_primer'][i])
    index_rv = oligo_name_list.index(golden_gate_assembly_2['rv_primer'][i])
    insert = anneal_oligos_object(oligo_list[index_fw],
                                  oligo_list[index_rv],
                                  pn2, pd2)
    annealed_DNA.append(insert)
    i = i+1

annealed_DNA


[<queen.QUEEN object; project='dna_102', length='29 bp', sequence='CACCGCCTTATGACCCTGACACGCTGTTT', topology='linear'>,
 <queen.QUEEN object; project='dna_106', length='29 bp', sequence='CACCGATTTAACTGGGCACGTTTGGGTTT', topology='linear'>,
 <queen.QUEEN object; project='dna_110', length='28 bp', sequence='CACCGGTTACCTCCACGTCGAACGGTTT', topology='linear'>,
 <queen.QUEEN object; project='dna_114', length='29 bp', sequence='CACCGAGCGTGTCAGGGTGACCGTGGTTT', topology='linear'>,
 <queen.QUEEN object; project='dna_118', length='29 bp', sequence='CACCGAGTCTGTCTCTCACAGCGTGGTTT', topology='linear'>,
 <queen.QUEEN object; project='dna_122', length='29 bp', sequence='CACCGAGTCTGGCAGTCACTGGGTGGTTT', topology='linear'>,
 <queen.QUEEN object; project='dna_126', length='29 bp', sequence='CACCGTGCGTGTCTCTGTCTCGGTGGTTT', topology='linear'>,
 <queen.QUEEN object; project='dna_130', length='29 bp', sequence='CACCGAGTCACGGAGCCAGTCGGTGGTTT', topology='linear'>,
 <queen.QUEEN object; project='dna_134', length='

In [10]:
#Step 3: Assemble
 
assembled_plasmids = []
i = 0
while i < len(golden_gate_assembly_2):
    assembly = joindna(annealed_DNA[i],
                       digested_backbones[i],
                       product = golden_gate_assembly_2['plasmid_id'][i],
                       topology = "circular", pn=pn3, pd=pd3)
    assembled_plasmids.append(assembly)
    i = i+1

assembled_plasmids[0].outputgbk()


LOCUS       pSI_827                 2975 bp    DNA     circular UNK 29-MAY-2024
DEFINITION  .
ACCESSION   pSI_827
VERSION     pSI_827
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
     source          1..1764
                     /organism="synthetic DNA construct"
                     /mol_type="other DNA"
                     /broken_feature="pSI_356_v2:0:4835:-:0..4835:1..1764"
     CDS             complement(462..1322)
                     /codon_start=1
                     /gene="bla"
                     /product="beta-lactamase"
                     /label="AmpR"
                     /note="confers resistance to ampicillin, carbenicillin, and
                     related antibiotics"
                     /translation="MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYI
                     ELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRIDAGQEQLGRRIHYSQNDLVEYS
                     PVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRW
     

In [11]:
#Step 4: Output QUEEN gbk file

i = 0
while i < len(golden_gate_assembly_2):
    assembled_plasmids[i].outputgbk(output = golden_gate_assembly_2['plasmid_id'][i]+ '.gbk')
    i = i+1


In [2410]:
#Step 5: Repair gbk locus ID (temporary fix; will be implemented in next update of QUEEN)

i = 0
while i < len(golden_gate_assembly_2):
    gbk_file = './' + golden_gate_assembly_2['plasmid_id'][i]+ '.gbk'
    handle = open(gbk_file, 'r')
    record = GenBank.read(handle)
    record.locus = golden_gate_assembly_2['plasmid_id'][i]
    record.accession = ''
    record.version = ''
    record.definition = 'Plasmid was generated by QUEEN v1.2.0 (https://github.com/yachielab/QUEEN)'
    record.organism = 'Synthetic DNA construct'
    fout = open(gbk_file, 'w') # final gbk file name
    fout.write(str(record))
    fout.close()
    i = i+1
    
print('QUEEN gbk files repaired.')

QUEEN gbk files repaired.
